In [1]:
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from scipy import stats
from scipy import optimize
import joblib
import matplotlib.pyplot as plt

#directory for results
filepath = ".../Resultate_final/PTF/IS/IS_test/PTF_VaR_ES_IS_1_test_saved/"

In [2]:
#Market and option parameters as in section 4.2 of 'Assessing Asset-Liability Risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
s_0 = 100
r = 0.01
corr= 0.3
tau = 1/52
T = 1/3
K = 100

mu = np.empty(20)
sigma = np.empty(20)
for i in range(0,10):
    mu[i] = mu[i+10] = (3+(i+1)/2)/100
    sigma[i] = sigma[i+10] = (15+(i+1))/100

cov_mat = np.empty((20,20))
for i in range(0,20):
    for j in range(0,20):
        if i != j:
            cov_mat[i,j] = corr
        else:
            cov_mat[i,j] = 1

C = np.linalg.cholesky(cov_mat)

#Confidence levels for Value-at-Risk and Expected Shortfall
alpha_VaR = 0.995
alpha_ES = 0.99

In [3]:
#Sizes for training set, validation set, test set, and set size for Monte Carlo estimation of the risk measures
M_1 = 1500000
M_2 = 500000
M_3 = 500000
#ignore N or N_2 in the following. Was kept just in case, but not used.
N_2 = 1
M_MC = 500000
#size of the set of data points used to calculate an IS density
M_IS = 2000000
#quantile for which the IS density will be computed
alpha_IS = 0.995

In [4]:
#Function for calculating simulated values of S_tau and simulated payoffs P_T from simulations of multivariate standard normal random variables
def data_gen(M,N,Z,V):
    #correlating the independent components
    Z = np.transpose(np.matmul(C,np.transpose(Z)))
    V = np.transpose(np.matmul(C,np.transpose(V)))
    
    #simulate S_tau under P
    S_tau_pre = np.empty((M, 20))
    for j in range(0,20):
        S_tau_pre[:,j] = s_0 * np.exp( (mu[j]-0.5*sigma[j]**2)*tau + np.sqrt(tau)*sigma[j]*Z[:,j] )
    S_tau = np.tile(S_tau_pre, (N,1))

    #simulate S_T given S_tau under Q
    S_T = np.empty((N*M,20))
    for j in range(0,20):
        S_T[:,j] = S_tau[:,j] * np.exp( (r-0.5*sigma[j]**2)*(T-tau) + np.sqrt(T-tau)*sigma[j]*V[:,j] )

    #compute discounted option payoffs
    P_T_pre =np.empty((len(S_T), 20))
    for j in range(0,10):
        P_T_pre[:,j] = np.exp(-r*(T-tau)) * np.maximum(S_T[:,j]-K,0)
    for j in range(10,20):
        P_T_pre[:,j] = np.exp(-r*(T-tau)) * np.maximum(K-S_T[:,j],0)
    P_T = np.sum(P_T_pre, axis=1)
    return S_tau,P_T

#the function DT(Z,\theta)
def data_trans_IS(Z,IS):
    res = np.empty((len(Z),20))
    for j in range(20):
        res[:,j] = Z[:,j]*np.sqrt(IS[20+j]) + IS[j]
    return res

#The density function of Z
def f(y):
    return stats.multivariate_normal.pdf(y, mean=np.full(20,0), cov=np.identity(20))

#The density function of Z_\theta (note that x is interpreted as theta, needed for the least-squares solver to work properly)
def f_theta(y, x):
    return stats.multivariate_normal.pdf(y, mean=x[0:20], cov=np.diag(x[20:40]), allow_singular=True)

#Put- und Call-Black-Scholes Formeln
def d1(K, t, x, sigma, r):
    return (np.log(x/K)+(r+0.5*sigma**2)*t)/(sigma*np.sqrt(t))

def d2(K, t, x, sigma, r):
    return (np.log(x/K)+(r-0.5*sigma**2)*t)/(sigma*np.sqrt(t))

def put_true(K, t, x, sigma, r): 
    return K*np.exp(-r*t)*stats.norm.cdf(-d2(K,t,x,sigma,r)) - x*stats.norm.cdf(-d1(K,t,x,sigma,r))

def call_true(K, t, x, sigma, r):
    return x*stats.norm.cdf(d1(K,t,x,sigma,r)) - K*np.exp(-r*t)*stats.norm.cdf(d2(K,t,x,sigma,r))

#true function l
def P_T_true(x):
    P_T = np.empty((len(x),20))
    for j in range(0,10):
        P_T[:,j] = call_true(K=100, t=(T-tau), x=x[:,j], sigma=sigma[j], r=r)
        P_T[:,j+10] = put_true(K=100, t=(T-tau), x=x[:,j+10], sigma=sigma[j+10], r=r)
    return np.sum(P_T, axis=1)

#This function describes the approximation of the expression inside the sum of m_2(theta)
def g_q_alpha_hat_reweighted(x,L,q_alpha_hat):
    return np.sqrt(f(y=L[:,0:20])/f_theta(y=L[:,0:20],x=x))*(L[:,-1]>q_alpha_hat)

#bounds for the IS density parameters (for the parameters corresponding to the mean no bounds are necessary, the standard deviation parameters however needs to be non-negative)
bnds_lower = np.empty((40))
bnds_upper = np.empty((40))
for j in range(20):
    bnds_lower[j] = -np.inf
    bnds_upper[j] = np.inf
    bnds_lower[20+j] = 0
    bnds_upper[20+j] = np.inf
    
bnds = (bnds_lower, bnds_upper)

In [5]:
#Generating realisations of multivariate standard normal random variables
Z_IS = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=M_IS)
V_IS = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=M_IS)

#Calculate the risk factor S_tau and the corresponding simulated payoffs P_T
S_tau_IS, P_T_IS = data_gen(M=M_IS, N=1, Z=Z_IS, V=V_IS)

#Calculate realisations of L_hat from the training data set using the true function l
L_hat_IS = np.column_stack((Z_IS, P_T_true(S_tau_IS)))
L_hat_IS_sort = L_hat_IS[L_hat_IS[:,-1].argsort()[::-1]]

#Calculating the corresponding estimator for Value-at-Risk in order to approximate g
q_alpha_hat_IS = L_hat_IS_sort[int(M_IS*(1-alpha_IS)-1), -1]
print('q_alpha_hat_IS:',q_alpha_hat_IS)

#Calculating the (hopefully) approximately optimal \theta^* by minimising m_2 using the approximated g
IS = optimize.least_squares(g_q_alpha_hat_reweighted, x0=np.concatenate((np.full(20,0),np.full(20,1))), args=(L_hat_IS, q_alpha_hat_IS), bounds=bnds).x

print(IS)

q_alpha_hat_IS: 110.2277654560932
[ 0.84624229  0.66549677  0.55264944  0.48526513  0.44432831  0.41710979
  0.38894521  0.3727674   0.37558848  0.37692719 -0.58150006 -0.56273828
 -0.54571362 -0.51313112 -0.50300692 -0.50788231 -0.48373343 -0.48509131
 -0.48892469 -0.47385691  1.71577507  1.41780252  1.31571827  1.23343008
  1.18449956  1.11683765  1.17822882  1.14104732  1.14259871  1.09038672
  1.07997994  1.10912008  1.08283955  1.11709199  1.10135617  1.10486008
  1.10710997  1.09278103  1.13279998  1.10300196]


In [7]:
run = 1
for j in range(100):
    #Generating simulations for multivariate standard normal random variables for  Monte Carlo estimation of risk measures
    Z_MC = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=M_MC)
    V_MC = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=M_MC)
    #calculating DT(Z,\theta^*)
    Z_MC_IS = data_trans_IS(Z_MC,IS)

    #Calculate the risk factor S_tau and the corresponding simulated payoffs P_T with original and IS distribution
    S_tau_MC,P_T_MC = data_gen(M=len(Z_MC),N=1,Z=Z_MC,V=V_MC)
    S_tau_MC_IS,P_T_MC_IS = data_gen(M=len(Z_MC),N=1,Z=Z_MC_IS,V=V_MC)
    
    #computation of option price depending on the risk factor S_tau according to the models, i.e. computation of L_hat_i's with original distribution
    L_hat = P_T_true(x=S_tau_MC)
    L_hat_sort = np.sort(L_hat)[::-1]
    
    #calculation of Value-at-Risk and Expected Shortfall estimators without IS
    j_VaR = int(M_MC*(1-alpha_VaR)-1)
    VaR_hat = L_hat_sort[j_VaR]
    
    j_ES = int(M_MC*(1-alpha_ES)-1)
    ES_hat = (1/(1-alpha_ES)) * np.sum(L_hat_sort[0:j_ES-1])/M_MC + ( 1 - (j_ES-1)/((1-alpha_ES)*M_MC) )*L_hat_sort[j_ES]
    
    #computation of option price depending on the risk factor S_tau according to the models, i.e. computation of L_hat_i's with IS distribution
    L_hat_IS = P_T_true(x=S_tau_MC_IS)
    L_hat_c_IS = np.column_stack((Z_MC_IS, L_hat_IS))
    
    #calculation of Value-at-Risk and Expected Shortfall estimators with IS
    L_hat_c_sort_IS = L_hat_c_IS[L_hat_c_IS[:,-1].argsort()[::-1]]
    w = f(L_hat_c_sort_IS[:,0:20])/(M_MC*f_theta(x=IS, y=L_hat_c_sort_IS[:,0:20]))

    j_VaR = 0
    w_sum_tmp = 0
    while (w_sum_tmp <= (1-alpha_VaR) and j_VaR<M_MC):
        w_sum_tmp += w[j_VaR]
        j_VaR += 1
    VaR_hat_IS = L_hat_c_sort_IS[j_VaR,-1]

    j_ES = 0
    w_sum_tmp = 0
    while (w_sum_tmp <= (1-alpha_ES) and j_ES<M_MC):
        w_sum_tmp += w[j_ES]
        j_ES += 1
    ES_hat_IS = (1/(1-alpha_ES)) * np.sum(w[0:j_ES-1]*L_hat_c_sort_IS[0:j_ES-1,-1]) + ( 1 - (1 / (1-alpha_ES)) * np.sum(w[0:j_ES-1]) )*L_hat_c_sort_IS[j_ES,-1]
    
    #save results for further evaluation
    output = np.array([VaR_hat,VaR_hat_IS,ES_hat,ES_hat_IS])
    
    joblib.dump(output, filepath+'output'+str(j)+'_'+str(run)+'.joblib')
    #prints just for checking while the notebook is running
    print(j)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
